**OCR For scanning and fetching the nutritional information**

In [ ]:
import cv2
import requests
import json
from tabulate import tabulate
from PIL import Image
from pyzbar.pyzbar import decode
import time

# =============== CONFIGURATION ===============
# Replace with your actual keys
USDA_API_KEY = "YOUR_API_KEY_HERE"  # Get a free key from https://fdc.nal.usda.gov/api-guide.html

# =============== API FUNCTIONS FOR NUTRITION ===============

def fetch_openfoodfacts_nutrition(barcode):
    """
    Fetches product name, ingredients, and detailed nutritional information
    from the OpenFoodFacts API.
    """
    url = f"https://world.openfoodfacts.org/api/v2/product/{barcode}.json"
    try:
        res = requests.get(url, timeout=10)
        res.raise_for_status()
        data = res.json()
        
        if data.get("status") == 1 and "product" in data:
            product = data["product"]
            nutriments = product.get("nutriments", {})
            
            nutrition_info = {
                "Calories (kcal)": nutriments.get("energy-kcal_100g"),
                "Fat (g)": nutriments.get("fat_100g"),
                "Saturated Fat (g)": nutriments.get("saturated-fat_100g"),
                "Carbohydrates (g)": nutriments.get("carbohydrates_100g"),
                "Sugars (g)": nutriments.get("sugars_100g"),
                "Protein (g)": nutriments.get("proteins_100g"),
                "Salt (g)": nutriments.get("salt_100g"),
                "Sodium (mg)": nutriments.get("sodium_100g", 0) * 1000 # Convert g to mg
            }
            
            # Filter out any keys with None values for a cleaner JSON
            nutrition_info = {k: v for k, v in nutrition_info.items() if v is not None}

            return {
                "source": "OpenFoodFacts",
                "barcode": barcode,
                "name": product.get("product_name", "Unknown Product"),
                "ingredients": product.get("ingredients_text_en", "Not specified"),
                "nutrition_per_100g": nutrition_info
            }
        return None
    except requests.RequestException as e:
        print(f"[-] OpenFoodFacts API error: {e}")
        return None

def fetch_usda_nutrition(barcode):
    """
    Fallback API: Fetches nutrition details from USDA FoodData Central.
    """
    if not USDA_API_KEY or USDA_API_KEY == "YOUR_USDA_API_KEY":
        return None  # Skip if no API key
        
    url = f"https://api.nal.usda.gov/fdc/v1/foods/search?query={barcode}&api_key={USDA_API_KEY}"
    try:
        res = requests.get(url, timeout=10)
        res.raise_for_status()
        data = res.json()
        
        if data.get("foods"):
            food = data["foods"][0]
            nutrients_map = {n['nutrientName']: n.get('value', "N/A") for n in food.get("foodNutrients", [])}
            
            nutrition_info = {
                "Calories (kcal)": nutrients_map.get("Energy"),
                "Fat (g)": nutrients_map.get("Total lipid (fat)"),
                "Carbohydrates (g)": nutrients_map.get("Carbohydrate, by difference"),
                "Sugars (g)": nutrients_map.get("Sugars, total including NLEA"),
                "Protein (g)": nutrients_map.get("Protein"),
                "Sodium (mg)": nutrients_map.get("Sodium, Na")
            }
            # Filter out any keys with None values
            nutrition_info = {k: v for k, v in nutrition_info.items() if v is not None}

            return {
                "source": "USDA FoodData Central",
                "barcode": barcode,
                "name": food.get("description", "Unknown Product"),
                "ingredients": food.get("ingredients", "Not specified"),
                "nutrition_per_100g": nutrition_info
            }
        return None
    except requests.RequestException as e:
        print(f"[-] USDA API error: {e}")
        return None

# =============== CAMERA + BARCODE (UNCHANGED) ===============
class FoodLabelScanner:
    def capture_from_camera(self):
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            print("Error: Could not open camera.")
            return None
        print("👉 Point camera at barcode... (Press 'q' to quit)")
        barcode = None
        while True:
            ret, frame = cap.read()
            if not ret: break
            
            decoded_objects = decode(Image.fromarray(frame))
            if decoded_objects:
                barcode = decoded_objects[0].data.decode("utf-8")
                cv2.putText(frame, f"Detected: {barcode}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            
            cv2.imshow("Scanning...", frame)

            if barcode:
                print(f"✅ Barcode Detected: {barcode}")
                time.sleep(2)
                break
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        cap.release()
        cv2.destroyAllWindows()
        return barcode

# =============== MAIN PIPELINE ===============
if __name__ == "__main__":
    scanner = FoodLabelScanner()
    barcode = scanner.capture_from_camera()

    if not barcode:
        print("\n❌ No barcode detected. Exiting.")
    else:
        print(f"\n🚀 Starting nutrition search for barcode: {barcode}\n")
        
        print("[1/2] Checking OpenFoodFacts API...")
        result = fetch_openfoodfacts_nutrition(barcode)
        
        if not result:
            print("[2/2] Checking USDA FoodData Central API...")
            result = fetch_usda_nutrition(barcode)

        print("\n" + "="*50)
        if result and result.get("nutrition_per_100g"):
            print(f"✅ Success! Found data from: {result['source']}\n")
            
            print(f"📦 Product: {result['name']}")
            print(f"🌿 Ingredients: {result['ingredients']}\n")

            nutrition_data = result["nutrition_per_100g"]
            table_data = [[key, value] for key, value in nutrition_data.items()]
            
            print("--- Nutrition Facts (per 100g) ---")
            print(tabulate(table_data, headers=["Nutrient", "Value"], tablefmt="grid"))

            # --- NEW: SAVE OUTPUT TO JSON FILE ---
            try:
                filename = f"{barcode}_data.json"
                with open(filename, 'w') as json_file:
                    json.dump(result, json_file, indent=4)
                print(f"\n💾 Data successfully saved to: {filename}")
            except Exception as e:
                print(f"\n❌ Error saving data to JSON file: {e}")

        else:
            print("❌ Failure: Could not retrieve nutritional information from any API.")
        print("="*50)

👉 Point camera at barcode... (Press 'q' to quit)
✅ Barcode Detected: 8886467124723

🚀 Starting nutrition search for barcode: 8886467124723

[1/2] Checking OpenFoodFacts API...

✅ Success! Found data from: OpenFoodFacts

📦 Product: Unknown Product
🌿 Ingredients: Not specified

--- Nutrition Facts (per 100g) ---
+-------------------+---------+
| Nutrient          |   Value |
+===================+=========+
| Calories (kcal)   | 541     |
+-------------------+---------+
| Fat (g)           |  30.9   |
+-------------------+---------+
| Saturated Fat (g) |  15     |
+-------------------+---------+
| Carbohydrates (g) |  57.6   |
+-------------------+---------+
| Sugars (g)        |   3.2   |
+-------------------+---------+
| Protein (g)       |   5     |
+-------------------+---------+
| Salt (g)          |   1.605 |
+-------------------+---------+
| Sodium (mg)       | 642     |
+-------------------+---------+

💾 Data successfully saved to: 8886467124723_data.json


**RuleBook and Normalization**

In [2]:
correct_json = {
  "reference_intakes": {
    "energy_kj": 8400,
    "energy_kcal": 2000,
    "fat": 70,
    "saturates": 20,
    "sugars": 90,
    "salt": 6
  },
  "thresholds": {
    "food": {
      "fat": { "green": "<=3", "amber": ">3 and <=17.5", "red": ">17.5" },
      "saturates": { "green": "<=1.5", "amber": ">1.5 and <=5", "red": ">5" },
      "sugars": { "green": "<=5", "amber": ">5 and <=22.5", "red": ">22.5" },
      "salt": { "green": "<=0.3", "amber": ">0.3 and <=1.5", "red": ">1.5" }
    },
    "drinks": {
      "fat": { "green": "<=1.5", "amber": ">1.5 and <=8.75", "red": ">8.75" },
      "saturates": { "green": "<=0.75", "amber": ">0.75 and <=2.5", "red": ">2.5" },
      "sugars": { "green": "<=2.5", "amber": ">2.5 and <=11.25", "red": ">11.25" },
      "salt": { "green": "<=0.3", "amber": ">0.3 and <=0.75", "red": ">0.75" }
    }
  },
  "weights": {
    "sugars": 0.40,
    "saturates": 0.25,
    "salt": 0.20,
    "fat": 0.15
  },
  "scores": { "green": 100, "amber": 50, "red": 0 },
  "bands": {
    "healthy": { "min": 70, "max": 100, "label": "Green Band", "description": "Healthy choice" },
    "moderate": { "min": 40, "max": 69, "label": "Amber Band", "description": "Moderate health profile" },
    "less_healthy": { "min": 0, "max": 39, "label": "Red Band", "description": "Less healthy choice" }
  },
  "evidence_sources": [
    "UK Government Front of Pack Nutrition Labelling Guidance (2016)",
    "EU Regulation No. 1169/2011, Annex XIII",
    "WHO recommendations on free sugars (<10% energy intake)"
  ]
}

import json
with open("scoring_rules.json", "w") as f:
    json.dump(correct_json, f, indent=2)


In [3]:
import os

print(os.getcwd())  # see current working directory
print(os.listdir()) # list all files


c:\Users\AMD\OneDrive\Desktop\Buildathon
['8904287001281_data.json', 'diet coke.json', 'FinalBackend.ipynb', 'FoP_Nutrition_labelling_UK_guidance.pdf', 'hide-and-seek_30170.jpg', 'ocr.ipynb', 'Packageratingapp.ipynb', 'panchratan mix.json', 'pasta macaroni.json', 'peanutbutter.json', 'rulebook_new.json', 'scoring_rules.json']


In [6]:
pip install google-generativeai


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import google.generativeai as genai

GEMINI_API_KEY = "YOUR_API_KEY_HERE"
genai.configure(api_key=GEMINI_API_KEY)


c:\Users\AMD\anaconda3\envs\Packagedproductapp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
def ask_gemini(prompt, model="gemini-1.5-flash"):
    try:
        model = genai.GenerativeModel(model)
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        return f"❌ Gemini API error: {e}"


**Scoring and commenting with LLM**

In [9]:
import json

def load_rules(path="scoring_rules.json"):
    with open(path, "r") as f:
        rules = json.load(f)
    return rules


In [10]:
def normalize_product_data(data):
    # Try to be flexible: nutrition_per_100g or normalized_nutrition
    nutrition = data.get("nutrition_per_100g") or data.get("normalized_nutrition") or {}

    mapping = {
        "sugars": ["Sugars (g)", "sugar", "sugars"],
        "saturates": ["Saturated Fat (g)", "saturates"],
        "salt": ["Salt (g)", "salt"],
        "fat": ["Fat (g)", "fat"],
    }

    normalized = {}
    for key, variants in mapping.items():
        for v in variants:
            if v in nutrition:
                normalized[key] = float(nutrition[v])
                break
        if key not in normalized:
            normalized[key] = 0.0  # default if missing

    return {
        "name": data.get("name", "Unknown"),
        "barcode": data.get("barcode", ""),
        "nutrition": normalized,
    }


In [11]:
import re

def classify_value(value, thresholds):
    for band, expr in thresholds.items():
        safe_expr = expr.strip()

        # Replace conditions like "<=5" with "value <= 5"
        safe_expr = re.sub(r"([<>]=?|==)\s*([\d\.]+)", r"value \1 \2", safe_expr)

        # Handle multiple conditions like ">3 and <=17.5"
        safe_expr = re.sub(r"\s+and\s+", " and ", safe_expr)

        try:
            if eval(safe_expr, {}, {"value": value}):
                return band
        except Exception as e:
            print(f"⚠️ Failed to eval '{expr}' for value={value}: {e}")

    return "unknown"



def score_product(product, rules, product_type="food"):
    nutrients = product["nutrition"]
    results = {}
    total_score = 0.0

    for n, val in nutrients.items():
        thresholds = rules["thresholds"][product_type][n]
        band = classify_value(val, thresholds)
        subscore = rules["scores"][band]
        weighted = subscore * rules["weights"][n]
        results[n] = {
            "value_per_100g": val,
            "band": band,
            "subscore": subscore,
            "weighted_score": weighted,
        }
        total_score += weighted

    # assign band
    band_label = None
    for band, info in rules["bands"].items():
        if info["min"] <= total_score <= info["max"]:
            band_label = info["label"]

    return {
        "product": product["name"],
        "barcode": product["barcode"],
        "score": round(total_score, 1),
        "band": band_label,
        "results": results,
        "evidence_sources": rules["evidence_sources"],
    }


In [ ]:
import google.generativeai as genai

genai.configure(api_key="YOUR_API_KEY_HERE")

def ask_gemini_comment(scored):
    prompt = f"""
    You are a nutrition assistant.
    Product: {scored['product']}
    Score: {scored['score']} ({scored['band']}).
    Nutrient breakdown:
    {json.dumps(scored['results'], indent=2)}

    Please give a short, consumer-friendly comment (2–3 sentences) about this product’s healthiness.
    """
    model = genai.GenerativeModel("gemini-1.5-flash")
    response = model.generate_content(prompt)
    return response.text


In [13]:
def evaluate_product(file_path, rules):
    with open(file_path, "r") as f:
        data = json.load(f)

    product = normalize_product_data(data)
    scored = score_product(product, rules, product_type="food")
    scored["llm_comment"] = ask_gemini_comment(scored)
    return scored


In [19]:
rules = load_rules("scoring_rules.json")

result = evaluate_product("8886467124723_data.json", rules)

print(json.dumps(result, indent=2))


{
  "product": "Unknown Product",
  "barcode": "8886467124723",
  "score": 40.0,
  "band": "Amber Band",
  "results": {
    "sugars": {
      "value_per_100g": 3.2,
      "band": "green",
      "subscore": 100,
      "weighted_score": 40.0
    },
    "saturates": {
      "value_per_100g": 15.0,
      "band": "red",
      "subscore": 0,
      "weighted_score": 0.0
    },
    "salt": {
      "value_per_100g": 1.605,
      "band": "red",
      "subscore": 0,
      "weighted_score": 0.0
    },
    "fat": {
      "value_per_100g": 30.9,
      "band": "red",
      "subscore": 0,
      "weighted_score": 0.0
    }
  },
  "evidence_sources": [
    "UK Government Front of Pack Nutrition Labelling Guidance (2016)",
    "EU Regulation No. 1169/2011, Annex XIII",
    "WHO recommendations on free sugars (<10% energy intake)"
  ],
  "llm_comment": "This product has a moderate overall health rating.  While the sugar content is low,  the high levels of saturated fat and salt are concerning and should b